<a href="https://colab.research.google.com/github/sidle34/allofthestuff/blob/Notebooks/SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRADTDA5622 - Big Data Computing Foundations 2
## Homework 2: Manipulating Data with SQL
- Semester: Spring 2025
- Instructor: Brad Coy
- Section: N/A
- Student Name: Erin Sidle
- Student Email: sidle.34@osu.edu
- Student ID: 500954191
***

***
# Section: Overview
***

**The Objectives of This Assignment are:**
1. To gain experience using SQL to manipulate data to solve problems and answer questions.

**Instructions:**
- **Follow the instructions** in each section.
- **Fill in** the **Conclusions** section.

***
# Section: Setup
- Add any needed imports, helper functions, etc., here.
- **I added all of these helper functions... Tom Bihari**
***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import sqlite3 as sq

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

data_directory = "/content/drive/MyDrive/State_data/"  # Set this to the appropriate directory for your setup.

In [ ]:
# Connect to a SQLite database (will be created if it doesn't exist).
def connect_db(file_name):
    if not os.path.isfile(file_name):
        print('Database file does not exist and will be created.')
    try:
        conn = sq.connect(file_name)
        print("Database connection established.")
        return conn
    except Error:
        print(Error)
        return None

In [ ]:
# Create a cursor to use to access the data.
def get_cursor(connection):
    try:
        cur = connection.cursor()
        print('Cursor created.')
        return cur
    except Error:
        print(Error)
        return None

In [ ]:
# Close the connection to the database.
def close_db(connection, cursor):
    connection.commit()
    cursor.close()
    connection.close()

In [ ]:
# Create a SQLite table by reading a CSV file.  Uses Pandas.
def create_table_from_csv(connection, file_name, table_name):
    pdf = pd.read_csv(file_name)
    create_table_from_dataframe(connection, pdf, table_name)

In [ ]:
# Create a SQLite table by reading a worksheet in an excel file.  Uses Pandas.
def create_table_from_excel(connection, file_name, sheet_name, table_name):
    pdf = pd.read_excel(file_name, sheet_name)
    create_table_from_dataframe(connection, pdf, table_name)

In [ ]:
# Create a SQLite table from a Pandas DataFrame.
def create_table_from_dataframe(connection, pdf, table_name):
    pdf.to_sql(table_name, connection, if_exists='replace', index = False)

In [ ]:
# Create a Pandas DataFrame from a SQLite cursor.
def create_dataframe_from_cursor(cursor):
    cols = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(data = cursor.fetchall(), columns = cols)

In [ ]:
# Create a Pandas DataFrame from a SQLite table (or view).
#   Or could do: df = pd.read_sql_query("SELECT * FROM table_name", cnx)
#   See: https://stackoverflow.com/questions/36028759/how-to-open-and-convert-sqlite-database-to-pandas-dataframe
def create_dataframe_from_table(connection, table_name):
    cursor = run_query(connection,'SELECT * FROM "' + table_name + '"')
    cols = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(data = cursor.fetchall(), columns = cols)

In [ ]:
# Run a SQLite query and return the result.
def run_query(conn_or_cur, query_string):
    try:
        result = conn_or_cur.execute(query_string)
    except:
        print("Query error")
        result = None
    return result

In [ ]:
# Replace text \n and \t with actual newline and tab characters.
def clean_string(strng):
    return str(strng).replace('\\n', '\n').replace('\\t', '\t')

In [ ]:
# Print query results up to a specified max number of records).
def print_result(cursor, max_num_records):
    print("------------------------------------------")
    df = create_dataframe_from_cursor(cursor)
    #print(df.head(max_num_records))
    display(df.head(max_num_records))
    print("------------------------------------------")

In [ ]:
def print_all_table_and_view_names(conn_or_cur):
    result = run_query(conn_or_cur,'SELECT name from sqlite_master where type= "table"')
    print("\nALL TABLE NAMES: ",result.fetchall())
    result = run_query(conn_or_cur,'SELECT name from sqlite_master where type= "view"')
    print("ALL VIEW NAMES:  ",result.fetchall())

In [ ]:
def print_schema(conn_or_cur, table_or_view_name):
    #https://www.sqlite.com/schematab.html
    # Table "sqlite_schema" is correct, but "sqlite_master" is used for backward compatibility.)
    result = run_query(conn_or_cur,'SELECT sql from sqlite_master where name= "' + table_or_view_name + '"')
    print("\nSCHEMA: " + table_or_view_name)
    print(clean_string(result.fetchall()))

In [ ]:
def print_table_or_view(conn_or_cur, table_name, max_num_records):
    print("\nTABLE/VIEW: " + table_name)
    result = run_query(conn_or_cur,'SELECT * FROM "' + table_name + '"')
    print_result(result, max_num_records)

---
# Set up a Database
- Create a blank database.
- Create three tables from the **state-abbrevs.csv**, **state-areas.csv**, and **state-populations.csv** files in the shared data folder.
- **I have done these steps for you**, from the SQL Examples notebook.
---

## Connect to a blank database

In [ ]:
# Connect to the database and get a cursor to access the data.
#conn = connect(r'SQLiteUSStatesData.db')  # a file database
conn = connect_db(r':memory:')             # an in-memory database
cur = get_cursor(conn)  # not needed for these examples, but kept for compatibility

Database file does not exist and will be created.
Database connection established.
Cursor created.


## Create some tables by reading data from CSV files

In [ ]:
create_table_from_csv(conn, data_directory + 'state-abbrevs.csv', 'abbrevs_table')
print_schema(conn,"abbrevs_table")
print_table_or_view(conn, "abbrevs_table", 3) # Show a few example records


SCHEMA: abbrevs_table
[('CREATE TABLE "abbrevs_table" (
"state" TEXT,
  "abbreviation" TEXT
)',)]

TABLE/VIEW: abbrevs_table
------------------------------------------


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ


------------------------------------------


In [ ]:
create_table_from_csv(conn, data_directory + 'state-areas.csv', 'areas_table')
#print_schema(conn,"areas_table")
print_table_or_view(conn, "areas_table", 3) # Show a few example records


TABLE/VIEW: areas_table
------------------------------------------


,state,area
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006


------------------------------------------


In [ ]:
create_table_from_csv(conn, data_directory + 'state-populations.csv', 'populations_table')
#print_schema(conn,"populations_table")
print_table_or_view(conn, "populations_table", 3) # Show a few example records


TABLE/VIEW: populations_table
------------------------------------------


,state,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0


------------------------------------------


In [ ]:
print_all_table_and_view_names(conn)


ALL TABLE NAMES:  [('abbrevs_table',), ('areas_table',), ('populations_table',)]
ALL VIEW NAMES:   []


---
# Task 1: Create a Table of Summarized State Populations
- Get the min, average, and max total and under18 populations for each of the states over all years.
- Call the table **summarized_state_populations**.
- The table should have the following fields:
  - state, ages, min_pop, avg_pop, max_pop
  - It should be sorted by **state** (from A to Z), and within each state, by **under18** age range first, then **total** age range.
---

In [ ]:
# Fill in the blanks
run_query(conn,'DROP TABLE IF EXISTS summarized_state_populations') # Delete the table if it already exists

run_query(conn,'''
    CREATE TABLE IF NOT EXISTS summarized_state_populations
    AS SELECT state, ages, MAX(population) as max_pop, AVG(population) as avg_pop, MIN(population) as min_pop
    FROM populations_table
    GROUP BY state, ages
    ORDER BY state, ages
    ''')


print_table_or_view(conn, "summarized_state_populations",3)



TABLE/VIEW: summarized_state_populations
------------------------------------------


,state,ages,max_pop,avg_pop,min_pop
0,AK,total,735132.0,6.462048e+05,553290.0
1,AK,under18,192636.0,1.866730e+05,177502.0
2,AL,total,4833722.0,4.484528e+06,4050055.0


------------------------------------------


---
# Task 2: Print the Top 3 States with the Largest Normalized Population Swings
- Consider the difference between max and min "total" populations for each state.
- Normalize by dividing by the average population for the state.
- You can use the table from the previous query.
---

In [ ]:
# Fill in the blanks

result = run_query(conn,'''
    SELECT state, MAX(max_pop) as total_max_pop, MIN(min_pop) as total_min_pop, (max_pop - min_pop) as difference
    FROM summarized_state_populations
    WHERE state NOT IN ('USA')
    GROUP BY state
    ORDER BY MAX(difference)DESC
    ''')

print_result(result, 3)

------------------------------------------


,state,total_max_pop,total_min_pop,difference
0,TX,26448193.0,4906220.0,9391438.0
1,CA,38332521.0,7980501.0,8373006.0
2,FL,19552860.0,2988807.0,6519553.0


------------------------------------------


---
# Task 3: Create a Table of Population Densities
- Follow each of the individual steps below.  There likely are shorter ways to do this task, but this way the steps are explicit.
- I have given the shells of some of the queries, so you can "fill in the blanks".
- I have included the expected outputs, so you can see if yours matches.
- Name the table **population_densities_table**
- The table should have the following fields:
  - state_abbrev
  - state_name
  - area
  - year
  - population_total
  - population_under_18
  - population_18_and_over
  - density_total
  - density_under_18
  - density_18_and_over
---

## Create a "pop_and_name_table"
- Join the populations_table and abbrevs_table.
- Table should have fields: state_abbrev, state_name, ages, year, population.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS pop_and_name_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS pop_and_name_table
    AS SELECT populations_table.state AS state_abbrev, abbrevs_table.state AS state_name, ages, year, population
    FROM (populations_table LEFT OUTER JOIN abbrevs_table ON populations_table.state = abbrevs_table.abbreviation)
    ''')

print_table_or_view(conn, "pop_and_name_table", 3) # Show a few records


TABLE/VIEW: pop_and_name_table
------------------------------------------


,state_abbrev,state_name,ages,year,population
0,AL,Alabama,under18,2012,1117489.0
1,AL,Alabama,total,2012,4817528.0
2,AL,Alabama,under18,2010,1130966.0


------------------------------------------


## Create a "pop_name_area_density_table"
- Join the pop_and_name_table and areas_table.
- Table should have fields: state_abbrev, state_name, ages, year, population, area, density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS pop_name_area_density_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS pop_name_area_density_table
    AS SELECT pop_and_name_table.*, areas_table.area, (population/area) as density
    FROM (pop_and_name_table LEFT OUTER JOIN areas_table ON pop_and_name_table.state_name = areas_table.state)
    ORDER BY year ASC, state_abbrev ASC, ages ASC
    ''')

print_table_or_view(conn, "pop_name_area_density_table", 3) # Show a few records


TABLE/VIEW: pop_name_area_density_table
------------------------------------------


,state_abbrev,state_name,ages,year,population,area,density
0,AK,Alaska,total,1990,553290.0,656425.0,0.842884
1,AK,Alaska,under18,1990,177502.0,656425.0,0.270407
2,AL,Alabama,total,1990,4050055.0,52423.0,77.257215


------------------------------------------


## Create an "under18_table"
- Select only the "under18" ages records from the pop_name_area_density_table.
- Table should have fields: state_abbrev, state_name, year, population, area, density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS under18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS under18_table
    AS SELECT state_abbrev, state_name, year, population, area, density
    FROM pop_name_area_density_table
    WHERE ages = 'under18'
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "under18_table", 3) # Show a few records


TABLE/VIEW: under18_table
------------------------------------------


,state_abbrev,state_name,year,population,area,density
0,AK,Alaska,1990,177502.0,656425.0,0.270407
1,AL,Alabama,1990,1050041.0,52423.0,20.030159
2,AR,Arkansas,1990,620933.0,53182.0,11.675623


------------------------------------------


## Create a "total_table"
- Select only the "total" ages records from the pop_name_area_density_table.
- Table should have fields: state_abbrev, state_name, year, population, area, density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS total_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_table
    AS SELECT state_abbrev, state_name, year, population, area, density
    FROM pop_name_area_density_table
    WHERE ages = "total"
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "total_table", 3) # Show a few records


TABLE/VIEW: total_table
------------------------------------------


,state_abbrev,state_name,year,population,area,density
0,AK,Alaska,1990,553290.0,656425.0,0.842884
1,AL,Alabama,1990,4050055.0,52423.0,77.257215
2,AR,Arkansas,1990,2356586.0,53182.0,44.311722


------------------------------------------


## Create a "total_and_under18_table"
- Join the under18_table and total_table.
- Table should have fields: state_abbrev, state_name, year, area, total_pop, total_density, under18_pop, under18_density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS total_and_under18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_and_under18_table
    AS SELECT total_table.state_abbrev, total_table.state_name, total_table.year, total_table.area,
        total_table.population as total_pop, total_table.density as total_density, under18_table.population as under18_pop, under18_table.density as under18_density
    FROM (total_table LEFT OUTER JOIN under18_table ON
        (total_table.state_abbrev = under18_table.state_abbrev AND total_table.year = under18_table.year))
    ''')

print_table_or_view(conn, "total_and_under18_table", 3) # Show a few records


TABLE/VIEW: total_and_under18_table
------------------------------------------


,state_abbrev,state_name,year,area,total_pop,total_density,under18_pop,under18_density
0,AK,Alaska,1990,656425.0,553290.0,0.842884,177502.0,0.270407
1,AL,Alabama,1990,52423.0,4050055.0,77.257215,1050041.0,20.030159
2,AR,Arkansas,1990,53182.0,2356586.0,44.311722,620933.0,11.675623


------------------------------------------


## Create a "total_under18_over18_table"
- Start with the total_and_under18_table and calculate columns for the over18_pop, over18_density.
- Actually, we mean "18 and over", not "over 18".
- Table should have fields: state_abbrev, state_name, year, area, total_pop, total_density, under18_pop, under18_density, over18_pop, over18_density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS total_under18_over18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_under18_over18_table
    AS SELECT *, (total_pop - under18_pop) as over18_pop, (total_density - under18_density) as over18_density
    FROM total_and_under18_table
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "total_under18_over18_table", 3) # Show a few records


TABLE/VIEW: total_under18_over18_table
------------------------------------------


,state_abbrev,state_name,year,area,total_pop,total_density,under18_pop,under18_density,over18_pop,over18_density
0,AK,Alaska,1990,656425.0,553290.0,0.842884,177502.0,0.270407,375788.0,0.572477
1,AL,Alabama,1990,52423.0,4050055.0,77.257215,1050041.0,20.030159,3000014.0,57.227057
2,AR,Arkansas,1990,53182.0,2356586.0,44.311722,620933.0,11.675623,1735653.0,32.636099


------------------------------------------


## Create the final population_densities_table"
- Start with the total_under18_over18_table.
- Rename and reorder the columns to match the original request.
- Sort by year and state abbreviation, ascending.
- Table should have fields: state_abbrev, state_name, area, year,
  - population_total, population_under_18, population_18_and_over, density_total, density_under_18, density_18_and_over.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS population_densities_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS population_densities_table
    AS SELECT state_abbrev, state_name, area, year, total_pop AS population_total, under18_pop as population_under_18, over18_pop as population_18_and_over,
        total_density AS density_total, under18_density as density_under_18, over18_density as density_18_and_over
    FROM total_under18_over18_table
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "population_densities_table", 10) # Show a few records


TABLE/VIEW: population_densities_table
------------------------------------------


,state_abbrev,state_name,area,year,population_total,population_under_18,population_18_and_over,density_total,density_under_18,density_18_and_over
0,AK,Alaska,656425.0,1990,553290.0,177502.0,375788.0,0.842884,0.270407,0.572477
1,AL,Alabama,52423.0,1990,4050055.0,1050041.0,3000014.0,77.257215,20.030159,57.227057
2,AR,Arkansas,53182.0,1990,2356586.0,620933.0,1735653.0,44.311722,11.675623,32.636099
3,AZ,Arizona,114006.0,1990,3684097.0,1006040.0,2678057.0,32.314940,8.824448,23.490492
4,CA,California,163707.0,1990,29959515.0,7980501.0,21979014.0,183.006927,48.748685,134.258242
5,CO,Colorado,104100.0,1990,3307618.0,881640.0,2425978.0,31.773468,8.469164,23.304304
6,CT,Connecticut,5544.0,1990,3291967.0,752666.0,2539301.0,593.789141,135.762266,458.026876
7,DC,District of Columbia,68.0,1990,605321.0,112632.0,492689.0,8901.779412,1656.352941,7245.426471
8,DE,Delaware,1954.0,1990,669567.0,165628.0,503939.0,342.664790,84.763562,257.901228
9,FL,Florida,65758.0,1990,13033307.0,2988807.0,10044500.0,198.201086,45.451610,152.749475


------------------------------------------


---
# Cleanup and Close the Current Database
---

In [ ]:
close_db(conn, cur)

---
# Task 4: Process Datasets with SQL and Answer Questions - From Scratch
In this Task, you will load several datasets into a new database.  You then will process the data and answer the following questions.  The code in Tasks 1-3 above can be used as examples.

The data is in an Excel file: 'MiscWikipediaCountryData.xls'.  You may open the file in Excel and look at it if you like.  The file contains four worksheets: GovernmentData, InternetData, IncomeEqualityData, and IncomePerCapitaData.

**Context and Hints**
- We are interested only in UN_Member countries.
- We don't care about the Year information.  We will assume that the data is relevant for the current year.
- Each dataset contains some of the information needed to answer the questions, so you will have to combine the information.  Consider building a single table "common_country_table" with columns:
  - **Country, Constitutional_Form, Head_Of_State, Subregion, Region, Population, Rich_Poor_Income_Ratio (use the CIA_RP10 column), Per_Capita_Income (use the IMF_Estimate column)**
  - Consider using JOIN operations.
- There is an issue with the data (as is usually the case...): Each dataset has a column that contains the country's name.  However, the datasets may contain different subsets of countries, and the country names may be spelled differently.  What should you do?  Here are some options:
  1. Treat one of the datasets (e.g., GovernmentData) as the primary dataset, and join the other datasets' data to the primary dataset IF you find a match; otherwise, join null information.
      - Pros: Gives a consistent set of countries.  Cons: Some fields will be null.
  2. Treat all of the datasets as "equal" and join information only if you find matching country names in all datasets.
      - Pros: No null fields, unless they were null in the original datasets.  Cons: A row will appear only if the country name appears in ALL datasets, so some countries will be missing!
  3. Do a deeper analysis and try to fix as many country name mismatches as possible.  Then join them.
      - Pros: May results in cleaner / more complete final product.  Cons: Difficult and probably manual work.
  4. Create a combined table of all of the country names across all of the datasets, and join all four datasets to that table.
      - Pros: Result will contain all information from all datasets.  Cons: A single real country may appear in multiple rows with different country names.

For simplicity, let's go with option 2.  What kind of JOIN accomplishes this?

To start things off, the initial setup has been given to you below.  Beneath that, you can fill in your own code.

---

## Connect to a blank database

In [ ]:
# Connect to the database and get a cursor to access the data.
conn = connect_db(r':memory:')             # an in-memory database
cur = get_cursor(conn)  # not needed for these examples, but kept for compatibility

Database file does not exist and will be created.
Database connection established.
Cursor created.


## Create four tables by reading data from the Excel file
- Review the Excel file iself to see the source and metadata for the datasets.
- Both .xls and .xlsx versions of the Excel file are provided in the data directory.  Some Python installations may not recognize the .xlsx version, so use what works for you.
- Create four tables: **govt_table**, **internet_table**, **income_eq_table**, **income_perc_table**. The govt_table is already done for you.

In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'GovernmentData', 'govt_table')
print_schema(conn,"govt_table")
print_table_or_view(conn, "govt_table", 3) # Show a few example records


SCHEMA: govt_table
[('CREATE TABLE "govt_table" (
"Name" TEXT,
  "UN_Member" INTEGER,
  "Constitutional_Form" TEXT,
  "Head_Of_State" TEXT,
  "Basis_Of_Executive_Legitimacy" TEXT
)',)]

TABLE/VIEW: govt_table
------------------------------------------


,Name,UN_Member,Constitutional_Form,Head_Of_State,Basis_Of_Executive_Legitimacy
0,Abkhazia,0,Republic,Executive,Presidency is independent of legislature
1,Afghanistan,1,Provisional,None,No constitutionally-defined basis to current regime[note 1]
2,Albania,1,Republic,Ceremonial,Ministry is subject to parliamentary confidence


------------------------------------------


In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'InternetData', 'internet_table')
print_schema(conn,"internet_table")
print_table_or_view(conn, "internet_table", 3) # Show a few example records


SCHEMA: internet_table
[('CREATE TABLE "internet_table" (
"Country_Or_Area" TEXT,
  "Subregion" TEXT,
  "Region" TEXT,
  "Internet_Users" INTEGER,
  "Pct" REAL,
  "Population" INTEGER,
  "Year" INTEGER
)',)]

TABLE/VIEW: internet_table
------------------------------------------


,Country_Or_Area,Subregion,Region,Internet_Users,Pct,Population,Year
0,Afghanistan,Southern Asia,Asia,4068194,0.101,40099462,2021
1,Albania,Southern Europe,Europe,2105339,0.737,2854710,2021
2,Algeria,Northern Africa,Africa,26350000,0.596,44177969,2021


------------------------------------------


In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'IncomeEqualityData', 'income_eq_table')
print_schema(conn,"income_eq_table")
print_table_or_view(conn, "income_eq_table", 3) # Show a few example records


SCHEMA: income_eq_table
[('CREATE TABLE "income_eq_table" (
"Country" TEXT,
  "Subregion" TEXT,
  "Region" TEXT,
  "UN_RP10" REAL,
  "UN_RP20" REAL,
  "CIA_RP10" REAL,
  "CIA_Year" REAL,
  "Gini_Percent" REAL,
  "Gini_Year" REAL
)',)]

TABLE/VIEW: income_eq_table
------------------------------------------


,Country,Subregion,Region,UN_RP10,UN_RP20,CIA_RP10,CIA_Year,Gini_Percent,Gini_Year
0,Afghanistan,Southern Asia,Asia,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,Southern Europe,Europe,7.2,4.2,7.2,2004.0,30.8,2019.0
2,Algeria,Northern Africa,Africa,9.6,4.0,9.6,1995.0,27.6,2011.0


------------------------------------------


In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'IncomePerCapitaData', 'income_perc_table')
print_schema(conn,"income_perc_table")
print_table_or_view(conn, "income_perc_table", 3) # Show a few example records


SCHEMA: income_perc_table
[('CREATE TABLE "income_perc_table" (
"Country_Or_Territory" TEXT,
  "UN_Region" TEXT,
  "IMF_Estimate" REAL,
  "IMF_Year" REAL,
  "WB_Estimate" REAL,
  "WB_Year" REAL,
  "UN_Estimate" REAL,
  "UN_Year" REAL
)',)]

TABLE/VIEW: income_perc_table
------------------------------------------


,Country_Or_Territory,UN_Region,IMF_Estimate,IMF_Year,WB_Estimate,WB_Year,UN_Estimate,UN_Year
0,Afghanistan,Asia,611.0,2020.0,369.0,2021.0,508.0,2020.0
1,Albania,Europe,6369.0,2022.0,6493.0,2021.0,5181.0,2020.0
2,Algeria,Africa,4151.0,2022.0,3691.0,2021.0,3368.0,2020.0


------------------------------------------


---
## Your Code
In the area below, write the SQL code to answer the following questions.  You may add / modify / delete cells as you wish.  Make sure you include comments that describe:
- What you are trying to do, and why.
- How you are doing it (comments in code, if appropriate).
- Your results or findings, if any.

**Answer These Questions:**
1. What is the Population of each Region?
2. Which Constitutional_Form has the highest Population, for counties that have reported their Population?
3. Which Constitutional_Form has the highest average Per_Capita_Income, for counties that have reported their Per_Capita_Income?
4. Which Country has the lowest average Rich_Poor_Income_Ratio, for counties that have reported their Rich_Poor_Income_Ratio?
5. What is the [Region, Subregion, Country, Population] of each of the 10 most populous countries in our dataset?  What is unrealistic about this answer?
6. What are the top 10 Countries in terms of total income (consider Population * Per_Capita_Income)?
---

In [ ]:
#Creating common_country_data data table

run_query(conn,'DROP TABLE IF EXISTS country_data') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS country_data
    AS SELECT govt_table.Name as Country, Constitutional_Form, Head_Of_State, Subregion, Region, COALESCE(Population,0) AS Population
    FROM (govt_table LEFT OUTER JOIN internet_table ON govt_table.Name = internet_table.Country_Or_Area)
    ''')

print_table_or_view(conn, "country_data", 3) # Show a few records


TABLE/VIEW: country_data
------------------------------------------


,Country,Constitutional_Form,Head_Of_State,Subregion,Region,Population
0,Abkhazia,Republic,Executive,None,None,0
1,Afghanistan,Provisional,None,Southern Asia,Asia,40099462
2,Albania,Republic,Ceremonial,Southern Europe,Europe,2854710


------------------------------------------


In [ ]:
run_query(conn,'DROP TABLE IF EXISTS country_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS country_table
    AS SELECT country_data.*, CIA_RP10 AS Rich_Poor_Income_Ratio
    FROM (country_data LEFT OUTER JOIN income_eq_table ON country_data.Country = income_eq_table.Country AND country_data.Subregion = income_eq_table.Subregion AND
    country_data.Region = income_eq_table.Region)
    WHERE CIA_RP10 IS NOT NULL
    ''')

print_table_or_view(conn, "country_table", 3) # Show a few records


TABLE/VIEW: country_table
------------------------------------------


,Country,Constitutional_Form,Head_Of_State,Subregion,Region,Population,Rich_Poor_Income_Ratio
0,Albania,Republic,Ceremonial,Southern Europe,Europe,2854710,7.2
1,Algeria,Republic,Executive,Northern Africa,Africa,44177969,9.6
2,Argentina,Republic,Executive,South America,Americas,45276780,35.0


------------------------------------------


In [ ]:
run_query(conn,'DROP TABLE IF EXISTS common_country_data') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS common_country_data
    AS SELECT country_table.*, IMF_Estimate AS Per_Capita_Income
    FROM (country_table LEFT OUTER JOIN income_perc_table ON country_table.Country = income_perc_table.Country_Or_Territory AND country_table.Region = income_perc_table.UN_Region)
    WHERE IMF_Estimate IS NOT NULL
    ''')

print_table_or_view(conn, "common_country_data", 3) # Show a few records


TABLE/VIEW: common_country_data
------------------------------------------


,Country,Constitutional_Form,Head_Of_State,Subregion,Region,Population,Rich_Poor_Income_Ratio,Per_Capita_Income
0,Albania,Republic,Ceremonial,Southern Europe,Europe,2854710,7.2,6369.0
1,Algeria,Republic,Executive,Northern Africa,Africa,44177969,9.6,4151.0
2,Argentina,Republic,Executive,South America,Americas,45276780,35.0,13622.0


------------------------------------------


In [ ]:
#Population of each region
run_query(conn,'DROP TABLE IF EXISTS region_population') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS region_population
    AS SELECT region, SUM(population) as Total_Population
    FROM common_country_data
    GROUP BY region
    ''')

print_table_or_view(conn, "region_population", 3) # Show a few records


TABLE/VIEW: region_population
------------------------------------------


,Region,Total_Population
0,Africa,1064452509
1,Americas,884842485
2,Asia,2822179810


------------------------------------------


In [ ]:
#Constitutional form with highest population

run_query(conn,'DROP TABLE IF EXISTS govt_pop') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS govt_pop
    AS SELECT Constitutional_Form, SUM(population) AS Total_Population
    FROM common_country_data
    GROUP BY Constitutional_Form
    ORDER BY Total_Population DESC
    ''')

print_table_or_view(conn, "govt_pop", 3) # Show a few records


TABLE/VIEW: govt_pop
------------------------------------------


,Constitutional_Form,Total_Population
0,Republic,4855613415
1,Constitutional monarchy,540380878
2,Provisional,144317297


------------------------------------------


In [ ]:
#Constitutional form with highest income per capita
run_query(conn,'DROP TABLE IF EXISTS govt_income') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS govt_income
    AS SELECT SUM(Per_Capita_Income) AS Income_Per_Capita, Constitutional_Form
    FROM common_country_data
    GROUP BY Constitutional_Form
    ORDER BY Income_Per_Capita DESC
    ''')

print_table_or_view(conn, "govt_income", 3) # Show a few records


TABLE/VIEW: govt_income
------------------------------------------


,Income_Per_Capita,Constitutional_Form
0,1272503.0,Republic
1,731017.0,Constitutional monarchy
2,5008.0,Provisional


------------------------------------------


In [ ]:
#Countries with the lowest rich/poor ratio that have reported a ratio

run_query(conn,'DROP TABLE IF EXISTS rich_poor_country') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS rich_poor_country
    AS SELECT Rich_Poor_Income_Ratio, Country
    FROM common_country_data
    GROUP BY Country
    ORDER BY Rich_Poor_Income_Ratio ASC
    ''')

print_table_or_view(conn, "rich_poor_country", 3) # Show a few records


TABLE/VIEW: rich_poor_country
------------------------------------------


,Rich_Poor_Income_Ratio,Country
0,4.5,Japan
1,5.2,Czech Republic
2,5.5,Bosnia and Herzegovina


------------------------------------------


In [ ]:
#The [Region, Subregion, Country, Population] of each of the 10 most populous countries in our dataset

run_query(conn,'DROP TABLE IF EXISTS most_populous') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS most_populous
    AS SELECT Country, Region, Subregion, SUM(Population) AS Total_Population
    FROM common_country_data
    GROUP BY Country
    ORDER BY population DESC
    ''')

print_table_or_view(conn, "most_populous", 10) # Show a few records


TABLE/VIEW: most_populous
------------------------------------------


,Country,Region,Subregion,Total_Population
0,India,Asia,Southern Asia,1407563842
1,United States,Americas,Northern America,336997624
2,Indonesia,Asia,South-eastern Asia,273753191
3,Brazil,Americas,South America,214326223
4,Pakistan,Asia,Southern Asia,213756286
5,Nigeria,Africa,Western Africa,213401323
6,Bangladesh,Asia,Southern Asia,166303498
7,Russia,Europe,Eastern Europe,145102755
8,Japan,Asia,Eastern Asia,124612530
9,Ethiopia,Africa,Eastern Africa,120283026


------------------------------------------


In [ ]:
#The top 10 Countries in terms of total income (consider Population * Per_Capita_Income)?

run_query(conn,'DROP TABLE IF EXISTS total_income') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_income
    AS SELECT Country, (population * Per_Capita_Income) AS Total_Income
    FROM common_country_data
    ORDER BY Total_Income DESC
    ''')

print_table_or_view(conn, "total_income", 10) # Show a few records


TABLE/VIEW: total_income
------------------------------------------


,Country,Total_Income
0,United States,2.533548e+13
1,Japan,4.281437e+12
2,Germany,4.036807e+12
3,India,3.471052e+12
4,United Kingdom,3.183604e+12
5,France,2.731616e+12
6,Canada,2.166976e+12
7,Russia,2.127932e+12
8,Italy,1.998769e+12
9,Brazil,1.898287e+12


------------------------------------------


---
# Cleanup and Close the Current Database
---

In [ ]:
close_db(conn, cur)

***
# Section: Conclusions
- What did you learn from this exercise?
***

In [ ]:
# I learned how to create tables and manipulate data using SQLite. I learned the similarities and differences between Python and SQLite. I learned that SQLite runs the FROM
#command before the AS SELECT command! I think that SQLite is a much simpler language than Python, but also has many capabilities and can be manipulated in many ways.